# Imports

In [1]:
# Import all libraries
import requests
from tqdm import trange
import zipfile
import os
from PIL import Image
import numpy as np
import pylab as plt
import matplotlib
from random import randint
from scipy.ndimage.interpolation import rotate
from skimage.transform import rescale, resize
from IPython.display import clear_output
import random
import shutil
import pandas as pd
import cv2
import time

%matplotlib inline

import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, Cropping2D, Reshape, BatchNormalization
from keras import optimizers, regularizers
from keras.optimizers import SGD, Adam
import keras.callbacks
from keras import backend as K

import h5py
from keras.utils.np_utils import to_categorical



from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam
from sklearn.metrics import accuracy_score, auc, roc_curve, roc_auc_score, confusion_matrix

from create_model import createResNet50, CreateKaggleModel

import stainNorm_Reinhard as stainNorm


# this part is needed if you run the notebook on Cartesius with multiple cores
#n_cores = 32
#config = tf.ConfigProto(intra_op_parallelism_threads=n_cores-1, inter_op_parallelism_threads=1, allow_soft_placement=True)
#session = tf.Session(config=config)
#K.set_session(session)
#os.environ["OMP_NUM_THREADS"] = str(n_cores-1)
#os.environ["KMP_BLOCKTIME"] = "1"
#os.environ["KMP_SETTINGS"] = "1"
#os.environ["KMP_AFFINITY"]= "granularity=fine,verbose,compact,1,0"

with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape = [2,3], name = 'a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape = [3,2], name = 'b')
c = tf.matmul(a, b)

sess = tf.Session(config=tf.ConfigProto(log_device_placement = True))
print(sess.run(c))



Using TensorFlow backend.


[[22. 28.]
 [49. 64.]]


# Classes

In [2]:
class DataSet:
    
    def __init__(self, data_dir_x, data_dir_y = None, use_percentage = 1.0):
        self.use_percentage = use_percentage
        self.data_dir_x = data_dir_x
        self.data_dir_y = data_dir_y
    
    def get_lenght(self):
        with h5py.File(self.data_dir_x, 'r') as hdf:
            return (int)(len(hdf['x'])*self.use_percentage)
    
    def get_data(self):
        with h5py.File(self.data_dir_x, 'r') as hdf:
            return np.array( list(hdf['x']))/255
        
    def get_lbls(self):
        with h5py.File(self.data_dir_y, 'r') as hdf:
            data = list(hdf['y'])
            return np.reshape(data,(len(data))) 
        
    
    def show_image(self, i):
        with h5py.File(self.data_dir_x, 'r') as hdf:
            data = list(hdf['x'][i])
            img = np.array(data)
            plt.rcParams['figure.figsize'] = (3, 3)
            plt.imshow(img)
            plt.title('RGB image')
            plt.show()

In [3]:
class Augmentor:
    def __init__(self, perc =.5):
        self.perc = perc
        
    def rotate(self, image):
        #rotation of 0-360
        rows,cols,channels = image.shape
        deg = np.random.randint(360)
        M = cv2.getRotationMatrix2D((cols/2,rows/2),deg,1)
        image = cv2.warpAffine(image,M,(cols,rows))
        return image
    
    def fliphorizontal(self, image):
        #horizontal flip
        image = cv2.flip(image, 0)
        return image
    
    def flipvertical(self, image):
        #vertical flip
        image = cv2.flip(image, 1)
        return image
    
    def flipboth(self, image):
        image = cv2.flip(image, -1)
        return image

In [4]:
class BatchCreator:
    
    def __init__(self, dataset, augmentor = None):
        self.dataset = dataset
        self.dataset_length = self.dataset.get_lenght()
        self.augmentor = augmentor
       
        self.index = []
        self.batch = []
        self.i = 0
        
        
    def create_batch(self, batch_size):
        x_data = np.array([])
        y_data = np.array([])
        
        if(self.i+batch_size > self.dataset_length):
            self.i = 0
      
        with h5py.File(self.dataset.data_dir_x, 'r') as hdf:
            data = np.array(list(hdf['x'][self.i:self.i+batch_size]))/255
                      
            if(self.augmentor != None):
                #augments data with random flip and rotation
                for i in range(len(data)):
                    data[i] = self.augmentor.rotate(data[i])
                    augtype = np.random.randint(1,4)
                    if(augtype == 1):
                        data[i] = self.augmentor.fliphorizontal(data[i])
                    elif(augtype == 2):
                        data[i] = self.augmentor.flipvertical(data[i])
                    elif(augtype == 3):
                        data[i] = self.augmentor.flipboth(data[i])
            x_data = data
        
        with h5py.File(self.dataset.data_dir_y, 'r') as hdf:
            data = list(hdf['y'][self.i:self.i+batch_size])
            data = np.reshape(data,(batch_size,1))
            data = to_categorical(data, num_classes=2)
            y_data = np.array(data) 
            
        self.i += batch_size
            
        return x_data, y_data
    

    def create_batch_sampling(self, batch_size):
        #init empty batch
        x_data = np.zeros( (batch_size, 96,96,3) )
        y_data = np.zeros( (batch_size, 2) )
        
        #copy the patches which have to be run again
        for i in range(len(self.index)):
            x_data[i] = self.batch[0][self.index[i]] 
            y_data[i] = self.batch[1][self.index[i]]
            
        #fill the rest of the batch with new patches
        rest = np.arange(len(self.index), batch_size)
        for i in rest:
            #get random position
            random_pos = np.random.randint(0,self.dataset_length)
            #open x file
            with h5py.File(self.dataset.data_dir_x, 'r') as hdf:
                data = np.array(list(hdf['x'][random_pos]))/255
                
                if(self.augmentor != None):
                    #augments data with random flip and rotation
                    data = self.augmentor.rotate(data)
                    augtype = np.random.randint(1,4)
                    if(augtype == 1):
                        data = self.augmentor.fliphorizontal(data)
                    elif(augtype == 2):
                        data = self.augmentor.flipvertical(data)
                    elif(augtype == 3):
                        data = self.augmentor.flipboth(data)
                                                           
                x_data[i] = data
                
                
            # open y file
            with h5py.File(self.dataset.data_dir_y, 'r') as hdf:
                data = list(hdf['y'][i])
                data = np.reshape(data,(1))
                data = to_categorical(data, num_classes=2)
                y_data[i] = data

        self.batch = [x_data, y_data]
            
        return x_data, y_data
    
        
    def get_generator(self, batch_size):
        '''returns a generator that will yield batches infinitely'''
        while True:
            yield self.create_batch(batch_size)
            
    def get_sampling_generator(self, batch_size):
        '''returns a generator that will yield batches infinitely'''
        while True:
            yield self.create_batch_sampling(batch_size)

In [5]:
def compute_accuracy(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

class Logger(keras.callbacks.Callback):

    def __init__(self, validation_data, saving_dir, model_name, batch_creator, sampling_threshold, sampling):
        #validation set
        self.validation_dataset = validation_data
        #traing batch generator
        self.batch_creator = batch_creator
        #threshold to for retraining a given sample
        self.sampling_threshold = sampling_threshold
        self.sampling = sampling
        self.time = time.time()
        self.model_name = model_name
        
        #get the validation data set
        self.val_imgs = self.validation_dataset.get_data()
        self.val_lbls = self.validation_dataset.get_lbls()
        
        self.Sensitivity = []
        self.Specificity = []
        self.auc = 0
        self.aucs = []
        
        self.model_filename = os.path.join(saving_dir, model_name + '.h5')
        
        self.losses = []
        self.accuracies = []
        self.best_validation_acc = 0
        self.best_model = None
        self.predictions = None
    
    def on_batch_end(self, batch, logs={}):
        if self.sampling:
            
            y_pred = self.model.predict(self.batch_creator.batch[0])
            y_true = self.batch_creator.batch[1][:,1]
            y_pred_argmax = np.argmax(y_pred, axis = 1)
            y_pred_max = np.max(y_pred, axis = 1)

            index = []
            for i in range(len(y_true)):
                # if the prediction is wrong add the retrain the sample
                if y_true[i] != y_pred_argmax[i]:
                    index.append(i)
                #else if the prediction is correct but below the threshold retrain the sample
                elif y_pred_max[i] < self.sampling_threshold:
                    index.append(i)
            self.batch_creator.index = index
    
        self.losses.append(logs.get('loss'))
        
    def on_epoch_end(self, epoch, logs={}):
        acc = self.validate()
        self.accuracies.append([len(self.losses), acc])
        if acc > self.best_validation_acc:
            self.best_validation_acc = acc
            self.model.save(self.model_filename) # save best model to disk
            print('best model saved as {}'.format(self.model_filename))
        self.plot()   
        
        t = time.time() - self.time
        hours = int(t/3600)
        t = t - hours*3600
        minutes = int(t/60)
        t = t - minutes*60
        seconds = t
    
        #print to logger file
        with open("./logs/current_status_{}.txt".format(self.model_name), "a") as myfile:
            myfile.write(str(epoch) + ' | ' + str(logs) + ' | val_acc = ' + str(acc)+' | auc = '+str(self.auc) 
                         +' | time = '+str(hours)+':'+str(minutes)+':'+str(seconds)+'\n')
        
    
    def validate(self):
        #predict validation set
        pred_lbls = self.model.predict(self.val_imgs)
        predicted_lbls = np.argmax(pred_lbls, axis=1)
        
        self.Specificity, self.Sensitivity, thresholds = roc_curve(self.val_lbls, pred_lbls[:,1])
        self.auc = roc_auc_score(self.val_lbls, pred_lbls[:,1])
        self.aucs.append(self.auc)
                    
        return compute_accuracy(self.val_lbls, predicted_lbls)

    
    def plot(self):
        clear_output()
        N = len(self.losses)
        plt.figure(figsize=(50, 10))
        plt.subplot(1, 4, 1)
        plt.plot(range(0, N), self.losses); plt.title('losses')
        plt.subplot(1, 4, 2)
        plt.plot(np.array(self.accuracies)); plt.title('accuracies')
        plt.subplot(1, 4, 3)
        plt.plot(np.array(self.aucs)); plt.title('AUC')
        plt.subplot(1, 4, 4)
        plt.plot(np.array(self.Specificity), np.array(self.Sensitivity)); 
        plt.title(' AUC = {}'.format(self.auc));
        plt.xlabel('1 - Specificity');
        plt.ylabel('Sensitivity')
        plt.show()

# Load Data

In [6]:
#data dirs
data_dir = './data/'
data_dir_norm = './data/normalized/'

#train_dir_x = os.path.join(data_dir_norm, 'x_train.h5')
#train_dir_y = os.path.join(data_dir, 'camelyonpatch_level_2_split_train_y.h5')
#valid_dir_x = os.path.join(data_dir_norm, 'x_valid.h5')
#valid_dir_y = os.path.join(data_dir, 'camelyonpatch_level_2_split_valid_y.h5')
#test_dir_x = os.path.join(data_dir_norm, 'x_test.h5')

train_dir_x = os.path.join(data_dir, 'camelyonpatch_level_2_split_train_x.h5')
train_dir_y = os.path.join(data_dir, 'camelyonpatch_level_2_split_train_y.h5')
valid_dir_x = os.path.join(data_dir, 'camelyonpatch_level_2_split_valid_x.h5')
valid_dir_y = os.path.join(data_dir, 'camelyonpatch_level_2_split_valid_y.h5')
test_dir_x = os.path.join(data_dir, 'camelyonpatch_level_2_split_test_x.h5')

print(train_dir_x)

./data/camelyonpatch_level_2_split_train_x.h5


# Test model

In [7]:
# load the best model
saving_dir = './models/'
model_name = "model_aug_15_epochs"
#model = createResNet50(in_t, True)
model = CreateKaggleModel(True)
model_path = os.path.join(saving_dir, model_name+'.h5')
model.load_weights(model_path)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 94, 94, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 92, 92, 32)        9216      
_________________________________________________________________
batch_normalization_1 (Batch (None, 92, 92, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 92, 92, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 46, 46, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 46, 46, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 44, 44, 64)        18432     
__________

In [9]:
training_dataset = DataSet(train_dir_x, train_dir_y, use_percentage=1.0)

batch_size = 64
n_steps = int(float(training_dataset.get_lenght())/batch_size)
print(n_steps)
print(float(training_dataset.get_lenght())/batch_size)

batch_creator = BatchCreator(training_dataset)
    
batch_generator = batch_creator.get_generator(batch_size)

4096
4096.0


In [10]:
predictions = model.predict_generator(batch_generator, steps = n_steps, verbose = 1)
print(predictions.shape)

4096/4096 [==============================] - 218s 53ms/step
(262144, 2)


In [11]:
lbls = training_dataset.get_lbls()
print(lbls)
print(lbls.shape)

[0 1 1 ... 1 0 1]
(262144,)


In [12]:
predicted_lbl = np.argmax(predictions, axis = 1)
print(predicted_lbl)
print(predicted_lbl.shape)

[0 1 1 ... 0 0 1]
(262144,)


In [14]:
learned = []
not_learned = []
for i in range(len(predicted_lbl)):
    if predicted_lbl[i] == lbls[i]:
        learned.append(i)
    else:
        not_learned.append(i)

print(len(learned))
print(len(not_learned))
    
    
    

235211
26933


In [15]:
pd_columns = ['learned']
df = pd.DataFrame(columns = pd_columns)
df['learned']       = learned


data_dir = "./data/"
if(not os.path.exists(data_dir)):
    os.mkdir(data_dir)

df.to_csv(os.path.join(data_dir,'learned.csv'), index=False)  

In [16]:
pd_columns = ['not_learned']
df = pd.DataFrame(columns = pd_columns)
df['not_learned']       = not_learned


data_dir = "./data/"
if(not os.path.exists(data_dir)):
    os.mkdir(data_dir)

df.to_csv(os.path.join(data_dir,'not_learned.csv'), index=False)  